## Cat vs. Dog

### Transfer Learning

In this notebook, you will learn to use pretrained network VGG-16 to extract image features from the cats and dog images. Then we will use a simple Multilayer perceptron to classify the images using the above extracted features as inputs.

The intuition behind transfer learning for image classification is that if a model is trained on a large and general enough dataset, We can then take advantage of these learned feature maps without having to start from scratch by training a large model on a large dataset.

## Data

The recommended folder structure is:  

### Folder structure

```python
dataset/
    train/ ###8000
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/ 
            cat001.jpg
            cat002.jpg
            ...
    test/ ###2000
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
```

### Data loading

In [1]:
from tensorflow import keras 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

ImageDataGenerator method in keras to feed all the images into model. ImageDataGenerator will automatically label the data and map all the labels to its specific data.

In [2]:
trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory=r"dataset/train",target_size=(224,224))

Found 8000 images belonging to 2 classes.


In [3]:
tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory(directory=r"dataset/test", target_size=(224,224))

Found 2000 images belonging to 2 classes.


## Using a pre-trained model

The process of training a convolutionnal neural network can be very time-consuming and require a lot of datas.  

In this tutorial we'll use VGG16, a model trained on the ImageNet dataset - which contains millions of images classified in 1000 categories. 

On top of it, we add a small multi-layer perceptron and we train it on our dataset.

## VGG-16 Network

The VGG-16 is a 16-layer network used by the VGG team in the ILSVRC-2014 competition. The network architecture and implementation details can be found in this [paper](https://arxiv.org/abs/1409.1556)

Though the VGG-16 model did not win the ILSVRC competition, the simplicity of the sequential model has led to it being used for a lot of transfer learning tasks.

Here in this part we'll import VGG16 from keras with pre-trained weights which was trained on imagenet. Here as you can see that include top parameter is set to true. This means that weights for our whole model will be downloaded. If this is set to false then the pre-trained weights will only be downloaded for convolution layers and no weights will be downloaded for dense layers.

In [4]:
from tensorflow.keras.applications.vgg16 import VGG16
vggmodel = VGG16(weights='imagenet', include_top=True)

2021-12-05 10:43:33.890025: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-05 10:43:33.959612: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2021-12-05 10:43:33.959646: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2021-12-05 10:43:33.960747: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN

553476096/553467096 [==============================] - 58s 0us/step


In [5]:
vggmodel.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

here will set trainable parameter to False for first 19 layers because we'll not be training the weights of the first 19 layers and use it as it is.

In [6]:
for layers in (vggmodel.layers)[:19]:
    print(layers)
    layers.trainable = False

Since our problem is to detect cats and dogs and it has two classes so the last dense layer of model should have 2 unit softmax dense layer. Also taking the second last layer of the model which is dense layer with 4096 units and adding a dense softmax layer of 2 units in the end to remove the last layer of the VGG16 model which is made to predict 1000 classes.

In [7]:
X= vggmodel.layers[-2].output
predictions = Dense(2, activation="softmax")(X)
model_final = Model(vggmodel.input, predictions)

In [8]:
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

/home/paresh/.cache/pypoetry/virtualenvs/upgrad-masters-ml-ai-7jz2oC-U-py3.8/lib/python3.8/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [9]:
model_final.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

ModelCheckpoint helps us to save the model by monitoring a specific parameter of the model. In this case we are monitoring validation accuracy by passing val_accuracy to ModelCheckpoint. The model will only be saved to disk if the validation accuracy of the model in current epoch is greater than what it was in the last epoch.

EarlyStopping helps us to stop the training of the model early if there is no increase in the parameter which we have set to monitor in EarlyStopping. In this case we are monitoring validation accuracy by passing val_accuracy to EarlyStopping.

In [10]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("model.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')
early = EarlyStopping(monitor='val_accuracy', patience=3, verbose=1, mode='auto')

Here we'll pass train and test data to fit_generator. In fit_generator steps_per_epoch will set the batch size to pass training data to the model and validation_steps will do the same for test data. tweak it based on your system specifications.

In [11]:
model_final.fit_generator(generator= traindata, steps_per_epoch= 2, epochs= 5, validation_data= testdata, validation_steps=1, callbacks=[checkpoint,early])

/tmp/ipykernel_16826/2316152776.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_final.fit_generator(generator= traindata, steps_per_epoch= 2, epochs= 5, validation_data= testdata, validation_steps=1, callbacks=[checkpoint,early])


Epoch 1/5
2/2 [==============================] - ETA: 0s - loss: 1.1170 - accuracy: 0.6562
Epoch 00001: val_accuracy improved from -inf to 0.75000, saving model to model.h5
2/2 [==============================] - 25s 19s/step - loss: 1.1170 - accuracy: 0.6562 - val_loss: 0.4728 - val_accuracy: 0.7500
Epoch 2/5
2/2 [==============================] - ETA: 0s - loss: 0.7489 - accuracy: 0.7031
Epoch 00002: val_accuracy improved from 0.75000 to 0.78125, saving model to model.h5
2/2 [==============================] - 25s 20s/step - loss: 0.7489 - accuracy: 0.7031 - val_loss: 0.4852 - val_accuracy: 0.7812
Epoch 3/5
2/2 [==============================] - ETA: 0s - loss: 0.2992 - accuracy: 0.8438
Epoch 00003: val_accuracy improved from 0.78125 to 0.87500, saving model to model.h5
2/2 [==============================] - 25s 19s/step - loss: 0.2992 - accuracy: 0.8438 - val_loss: 0.2958 - val_accuracy: 0.8750
Epoch 4/5
2/2 [==============================] - ETA: 0s - loss: 0.4294 - accuracy: 0.8438


In [12]:
test_loss, test_acc = model_final.evaluate_generator(testdata, steps=2)

/tmp/ipykernel_16826/2484837891.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  test_loss, test_acc = model_final.evaluate_generator(testdata, steps=2)


In [13]:
print('test acc:', test_acc)
print('test loss:', test_loss)

test acc: 0.890625
test loss: 0.22538495063781738
